In [1]:
import requests
import re

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

from time import sleep

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [6]:
lista_actores = ['Adam Sandler', 'Byom', 'Rose LaRue', 'Aaron Jackson', 'Adam Sandler', 'Deborah Smith Ford','Jason Menz']
lista_detalles_actores = []
datos_actor = {}
url_actores=(f"https://www.imdb.com/")
driver = webdriver.Chrome()
driver.get(url_actores)
driver.maximize_window()
sleep(2)
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
sleep(3)
for actor in lista_actores:
    driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)
    sleep(5)
    datos_actor["nombre_actor"] = actor
    driver.execute_script("window.scrollTo(0,400)")
    sleep (2)
    driver.find_element('xpath','//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div/a').click()
    sleep (3)
    # try:
    #     driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/ul/li[2]/a').click()
    #     driver.execute_script("window.scrollTo(0,400)")
    #     texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section/div/section/div/div[1]/div[2]/div/div').text
    #     resultado_premios = re.search(r'(\d+)\s*premio[s]?',texto_premios, re.IGNORECASE).group(1)    
    #     datos_actor['premios'] = resultado_premios
    # except:
    #     print("Error al extrear los premios")
    #     datos_actor['premios'] = "0"
    # sleep(2)   
    # driver.back()
    # sleep(2)
    try: 
        resultado_que_hace = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
        lista_que_hace = resultado_que_hace.split("\n")
        datos_actor["que_hace"] = lista_que_hace
    except:
        print("Error al encontrar la seccion 'que hace'")
        datos_actor["que_hace"] = "None"
    sleep(2)
    try:
        driver.execute_script("window.scrollTo(0,500)")
        anno_nacimiento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text
        resultado_anno_nacimiento = re.search(r'\b\d{4}\b', anno_nacimiento).group()
        datos_actor["anno_nacimiento"] = resultado_anno_nacimiento
        #anno_nacimiento antiguo:
        #datos_actor["anno_nacimiento"] = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text.split(" ")[-1]
    except:
        print("Error al extrear el año de nacimiento")
        datos_actor['anno_nacimiento'] = "0"            #PREMIO ANTIGUOS: //*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li
    sleep(3)                                            #//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul/li/span
    try:                                                #//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul
        texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li').text
        #.split(" ")[0]
        resultado_premios = re.search(r'(\d+)\s+premios',texto_premios, re.IGNORECASE).group(1)
        datos_actor['premios'] = resultado_premios
    except:
        print("Error al extrear los premios")
        datos_actor['premios'] = "0"
    sleep(4)
    listas_pelis_conocido_por = []
    for i in range(1,5):
        try:
            driver.execute_script("window.scrollTo(0,1500)")
            sleep(3)
            resultado_conocido_por = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a').text
            listas_pelis_conocido_por.append(resultado_conocido_por.split("\n")[0])            
        except:
            print("Error al encontrar las peliculas")
            listas_pelis_conocido_por.append("None")
    sleep(2)
    datos_actor["peliculas"] = listas_pelis_conocido_por    
    # try:
    #     tupla_actor = (datos_actor['nombre_actor'], int(datos_actor['anno_nacimiento']), datos_actor["peliculas"], datos_actor["que_hace"], int(datos_actor['premios'])) 
    # except:
    #     print("Error al crear la tupla. Creamos tupla con null")
    #     tupla_actor = (datos_actor['nombre_actor'], 0, ["None"], ["None"], 0)
    
    lista_detalles_actores.append(datos_actor)
    datos_actor = {}
        
print("----------------")
    
print(lista_detalles_actores)

Error al extrear los premios
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al extrear el año de nacimiento
Error al extrear los premios
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al extrear el año de nacimiento
Error al extrear los premios
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al extrear el año de nacimiento
Error al extrear los premios
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al extrear los premios
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al extrear el año de nacimiento
Error al extrear los premios
Error al encon

In [ ]:
#anno nacimiento funcionante

anno_nacimiento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text.split(" ")[-1]

In [36]:
#"10 de julio de 19??"
anno_nacimiento = "11 de noviembre de 1994"
resultado = re.search(r'\b\d{4}\b', anno_nacimiento).group()
print(resultado)


1994


In [ ]:
lista_actores = ['Byom', 'Rose LaRue', 'Aaron Jackson', 'Adam Sandler', 'Deborah Smith Ford','Jason Menz']

In [47]:
import pandas as pd

In [51]:
df = pd.DataFrame(lista_detalles_actores)
df

,nombre_actor,que_hace,anno_nacimiento,premios,peliculas
0,Byom,"[Reparto, Producción, Dirección de la segunda ...",0,0,"[Mom's Coming, Lage Na Mon, Love Driver, Jai H..."
1,Rose LaRue,"[Reparto, Assistant, Guion y continuidad]",0,0,"[Bloodline, Forgiven, Love Song and Power, Eve..."
2,Aaron Jackson,"[Reparto, Guion, Producción]",0,0,"[Rabos: El musical, The Chris Gethard Show: Pu..."
3,Adam Sandler,"[Reparto, Producción, Guion]",1966,76,"[El aguador, Little Nicky, El chico ideal, Un ..."
4,Deborah Smith Ford,"[Reparto, Guion, Producción]",19??,0,"[Listen like it's Gospel, Mickey, Love Song an..."
5,Jason Menz,"[Reparto, Equipos adicionales, Producción]",0,0,"[The Young and the Restless, RockSlide, Todd, ..."


In [5]:
lista_id_peliculas = ['tt10579986','tt10597228','tt11097384']
lista_actores = []
url_actores=("https://www.imdb.com/")
driver = webdriver.Chrome()
driver.get(url_actores)
driver.maximize_window()
sleep(2)
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
sleep(2)
for pelicula in lista_id_peliculas:
    print(pelicula)
    try:
        driver.find_element("css selector", "#suggestion-search").send_keys(pelicula, Keys.ENTER)
    except:
        print("Pelicula no encontrada")
    sleep(2)
    #driver.execute_script("window.scrollTo(0,1000)")
    for i in range(1,11):
        try:       
            actor = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i})').text.split("\n")[0]
            lista_actores.append(actor)   
        except:
            print(f"Error al encontrar los 10 primeros actores de la pelicula {pelicula}") 
    sleep(2)
    driver.back()
    sleep(2)
print(lista_actores)        

tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
Error al encontrar los 10 primeros actores de la pelicula tt10579986
tt10597228
tt11097384
['Byom', 'Aaron Jackson', 'Becky Brown', 'Cassidy Reyes', 'Rose LaRue', 'Mercedes Gutierrez', 'Deborah Smith Ford', 'Tay Lindsey', 'Jason Menz', 'Marc Forman', 'Colleen Winfough', 'Adam Sandler', 'Carey Mulligan', 'Paul Dano', 'Kunal Nayyar', 'Isabella Rossellini', 'Lena Olin', 'Petr Papánek', 'Marian Roden', 'Zuzana Stivínová', 'Sinead Phelps'

In [ ]:
#FUNCIONAAAAA
actor = driver.find_element('xpath',f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]/div[{i}]').text.split("\n")[0]

actor = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i})').text.split("\n")[0]